In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
def read_data():    
    dataset = {}
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            data_path = os.path.join(dirname, filename)
            if filename.split('.')[0] == 'train':
                dataset['train']= pd.read_csv(data_path)
            if filename.split('.')[0] == 'test':
                dataset['test'] = pd.read_csv(data_path)
            else:
                dataset['sample'] = pd.read_csv(data_path)
    return dataset

def data_preprocessing(dataset):         
    # print(dataset['train'].head)
    # sns.countplot(x='Survived', data = dataset['train'])
    # print(dataset['train'].shape)
    dataset.drop(['PassengerId','Name','Ticket','Cabin','Embarked'], axis = 1, inplace = True)
    dataset['Sex'].replace({'male':0, 'female':1}, inplace = True)
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
#     print(dataset.isna().any())
    
    
    X = dataset.loc[:,dataset.columns != 'Survived']
    # print(X.shape)
    y = dataset.loc[:,dataset.columns == 'Survived']
    # print(y.shape)
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.33, random_state = 69)
#     for data in X_train:
#         print(X_train)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    y_train = y_train.to_numpy()
    t_test = y_test.to_numpy()
    # print(X_train)
    return (X_train, X_test, y_train, y_test)

dataset = read_data()
(X_train, X_test,y_train, y_test) = data_preprocessing(dataset['train'])
# data_preprocessing(dataset['train'])
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


(596, 6) (295, 6) (596, 1) (295, 1)


In [2]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train))
print(type(y_train))
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__(self):
        return len(self.X_data)
    
class TestData(Dataset):
    def __init__(self, X_data):
        self.X_data = X_data
    def __getitem__(self, index):
        return self.X_data[index]
    def __len__(self):
        return len(self.X_data)
    
train_data = TrainData(torch.FloatTensor(X_train), 
                      torch.FloatTensor(y_train))
test_data = TestData(torch.FloatTensor(X_test))

train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = 1)

# for X,y in train_loader:
#     print (X)

print(train_loader, test_loader)

(596, 6) (295, 6) (596, 1) (295, 1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<torch.utils.data.dataloader.DataLoader object at 0x7f076dd80e10> <torch.utils.data.dataloader.DataLoader object at 0x7f076dd74350>


In [3]:
class BinaryClassification(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dims):
        super(BinaryClassification, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dims = hid_dims
        current_dim = in_dim
        self.layers = nn.ModuleList()
        for hdim in hid_dims:
            self.layers.append(nn.Linear(current_dim, hdim))
            self.layers.append(nn.BatchNorm1d(hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, out_dim))
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
#         self.batchnorms = []
#         for hdim in hid_dims:
#             self.batchnorms.append(nn.BatchNorm1d(hdim))
        
    def forward(self, inputs):
        idx = 0
        x = inputs
        for layer in self.layers[:-1]:
#             print ('input ', x)
            x = self.relu(layer(x))
#             x= self.batchnorms[idx](x)
            idx =idx + 1
#             print('output ',x)
#         print ('input', x)
        x = self.dropout(x)
        x = self.layers[-1](x)
#         print ('output', x)
        return x
                

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = BinaryClassification(6, 1, [12,12,12])
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

cpu
BinaryClassification(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=12, out_features=12, bias=True)
    (3): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=12, out_features=12, bias=True)
    (5): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=12, out_features=1, bias=True)
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)


In [5]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)  
    return acc

model.train()
EPOCHS = 40
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
#         print (X_batch)
        y_pred = model(X_batch)
#         print (y_pred)
#         print(y_batch)
        loss = criterion(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.68769 | Acc: 60.300
Epoch 002: | Loss: 0.66294 | Acc: 62.700
Epoch 003: | Loss: 0.64589 | Acc: 65.400
Epoch 004: | Loss: 0.63675 | Acc: 65.800
Epoch 005: | Loss: 0.61726 | Acc: 71.000
Epoch 006: | Loss: 0.61386 | Acc: 68.600
Epoch 007: | Loss: 0.60485 | Acc: 70.600
Epoch 008: | Loss: 0.59611 | Acc: 72.700
Epoch 009: | Loss: 0.59782 | Acc: 71.500
Epoch 010: | Loss: 0.57521 | Acc: 75.800
Epoch 011: | Loss: 0.57268 | Acc: 75.500
Epoch 012: | Loss: 0.56820 | Acc: 75.200
Epoch 013: | Loss: 0.56059 | Acc: 76.200
Epoch 014: | Loss: 0.53901 | Acc: 77.100
Epoch 015: | Loss: 0.54167 | Acc: 77.900
Epoch 016: | Loss: 0.53077 | Acc: 77.200
Epoch 017: | Loss: 0.52756 | Acc: 77.900
Epoch 018: | Loss: 0.51127 | Acc: 79.600
Epoch 019: | Loss: 0.50566 | Acc: 78.900
Epoch 020: | Loss: 0.51538 | Acc: 79.500
Epoch 021: | Loss: 0.50031 | Acc: 79.100
Epoch 022: | Loss: 0.49372 | Acc: 79.900
Epoch 023: | Loss: 0.47077 | Acc: 82.700
Epoch 024: | Loss: 0.47995 | Acc: 81.600
Epoch 025: | Los

In [6]:
y_pred_list = []
# print(model)
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
#         print(X_batch)
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]


In [7]:
confusion_matrix(y_test, y_pred_list)
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       187
           1       0.77      0.67      0.71       108

    accuracy                           0.80       295
   macro avg       0.79      0.77      0.78       295
weighted avg       0.80      0.80      0.80       295



In [8]:
def testdata_processing(dataset):
#     print(dataset)
    data = dataset.drop(['PassengerId','Name','Ticket','Cabin','Embarked'], axis = 1)
    data['Sex'].replace({'male':0, 'female':1}, inplace = True)
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['Fare'].fillna(data['Age'].mean(), inplace = True)
    print(data.isna().any())
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return data

In [9]:
data = None
data = testdata_processing(dataset['test'])
# print(data)
val_data = TestData(torch.FloatTensor(data))

Pclass    False
Sex       False
Age       False
SibSp     False
Parch     False
Fare      False
dtype: bool


In [10]:
print(val_data)

In [11]:
val_loader = DataLoader(dataset = val_data, batch_size = 1)

In [12]:
y_pred_list = []
# print(model)
model.eval()
with torch.no_grad():
    for X_batch in val_loader:
#         print(X_batch)
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
print(y_pred_list)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,

In [13]:
data_X = dataset['test']
print(type(data_X))
data_y = y_pred_list
data = []
for index, row in data_X.iterrows():
    data.append([row['PassengerId'], int(data_y[index])])
#     print (row['PassengerId'], int(data_y[index]))
df = pd.DataFrame(data, columns = ['PassengerId', 'Survived'])
df.to_csv("predictions.csv", index = False)

<class 'pandas.core.frame.DataFrame'>
